In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 3
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC10(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC10(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC10(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
讀取3秒資料
讀取5秒資料
Data Loading Time: 0.20 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        
        #------------------------------------------------
        #需要調整的地方
        #------------------------------------------------
        #--要用幾秒訓練 data_1S/data_3S/data_1S
        #--學習率
        #--epoch
        if(TL==0):
            trainData = data_1S
        elif(TL==1):
            trainData = data_3S
        else:
            trainData = data_5S
        #------------------------------------------------
    

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型
        model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold)
    
        #測試準確度
        test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
        test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
        test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1


C:\Users\test\anaconda3\envs\pytorch\lib\site-packages\torchvision\ops\misc.py:120: UserWarning: Don't use ConvNormActivation directly, please use Conv2dNormActivation and Conv3dNormActivation instead.
  warnings.warn(
Downloading: "https://github.com/fschmid56/EfficientAT/releases/download/v0.0.1/mn40_as_ext_mAP_487.pt" to resources\mn40_as_ext_mAP_487.pt
22.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

32.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_

Number of classes defined: 10, but try to load pre-trained layer with logits: 527
Dropping last layer.
Error(s) in loading state_dict for MobileNetV3:
	Missing key(s) in state_dict: "classifier.5.weight", "classifier.5.bias". 
Loading weights pre-trained weights in a non-strict manner.
MobileNetV3(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=Fa

----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------


C:\Users\test\anaconda3\envs\pytorch\lib\site-packages\torch\functional.py:641: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\SpectralOps.cpp:867.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


...

Train Epoch: 1/50 Train_Loss: 0.006976352073252201 Train_acc: 100.000000%
Train Epoch: 2/50 Train_Loss: 0.007465910166501999 Train_acc: 100.000000%
Train Epoch: 3/50 Train_Loss: 0.01217450387775898 Train_acc: 100.000000%
Train Epoch: 4/50 Train_Loss: 0.01335871685296297 Train_acc: 100.000000%
Train Epoch: 5/50 Train_Loss: 0.009198678657412529 Train_acc: 100.000000%
Train Epoch: 6/50 Train_Loss: 0.013243312016129494 Train_acc: 100.000000%
Train Epoch: 7/50 Train_Loss: 0.009152901358902454 Train_acc: 100.000000%
Train Epoch: 8/50 Train_Loss: 0.004075620789080858 Train_acc: 100.000000%
Train Epoch: 9/50 Train_Loss: 0.007128020748496056 Train_acc: 100.000000%
Train Epoch: 10/50 Train_Loss: 0.00337030622176826 Train_acc: 100.000000%
Train Epoch: 11/50 Train_Loss: 0.004998982883989811 Train_acc: 100.000000%
Train Epoch: 12/50 Train_Loss: 0.0027291509322822094 Train_acc: 100.000000%
Train Epoch: 13/50 Train_Loss: 0.005298989824950695 Train_acc: 100.000000%
Train Epoch: 14/50 Train_Loss: 0.004

----------------------------------------------------------------------------------------------------------------
fold:  4 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/50 Train_Loss: 2.3282854557037354 Train_acc: 14.562500%
Train Epoch: 2/50 Train_Loss: 2.2961807250976562 Train_acc: 13.500000%
Train Epoch: 3/50 Train_Loss: 2.2334437370300293 Train_acc: 16.000000%
Train Epoch: 4/50 Train_Loss: 2.240281820297241 Train_acc: 16.187500%
Train Epoch: 5/50 Train_Loss: 2.171041965484619 Train_acc: 23.000000%
Train Epoch: 6/50 Train_Loss: 2.0590662956237793 Train_acc: 31.687500%
Train Epoch: 7/50 Train_Loss: 1.9403032064437866 Train_acc: 48.500000%
Train Epoch: 8/50 Train_Loss: 1.6824852228164673 Train_acc: 65.562500%
Train Epoch: 9/50 Train_Loss: 1.3663743734359741 Train_acc: 76.000000%
Train Epoch: 10/50 Train_Loss: 1.0573549270629883 Train_acc: 80.250000%
Train Epoch: 11/50 Train_Loss: 0.797602653503418 

Train Epoch: 1/50 Train_Loss: 0.004037322010844946 Train_acc: 100.000000%
Train Epoch: 2/50 Train_Loss: 0.004973972216248512 Train_acc: 100.000000%
Train Epoch: 3/50 Train_Loss: 0.020802486687898636 Train_acc: 100.000000%
Train Epoch: 4/50 Train_Loss: 0.006292479578405619 Train_acc: 100.000000%
Train Epoch: 5/50 Train_Loss: 0.007014888804405928 Train_acc: 100.000000%
Train Epoch: 6/50 Train_Loss: 0.007211498916149139 Train_acc: 100.000000%
Train Epoch: 7/50 Train_Loss: 0.006589408963918686 Train_acc: 100.000000%
Train Epoch: 8/50 Train_Loss: 0.005125200375914574 Train_acc: 100.000000%
Train Epoch: 9/50 Train_Loss: 0.008360132575035095 Train_acc: 100.000000%
Train Epoch: 10/50 Train_Loss: 0.008793547749519348 Train_acc: 100.000000%
Train Epoch: 11/50 Train_Loss: 0.005809125956147909 Train_acc: 100.000000%
Train Epoch: 12/50 Train_Loss: 0.0041713775135576725 Train_acc: 100.000000%
Train Epoch: 13/50 Train_Loss: 0.0028466880321502686 Train_acc: 100.000000%
Train Epoch: 14/50 Train_Loss: 0

----------------------------------------------------------------------------------------------------------------
fold:  5 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/50 Train_Loss: 2.3599913120269775 Train_acc: 7.375000%
Train Epoch: 2/50 Train_Loss: 2.317896604537964 Train_acc: 7.500000%
Train Epoch: 3/50 Train_Loss: 2.288177490234375 Train_acc: 8.000000%
Train Epoch: 4/50 Train_Loss: 2.301966428756714 Train_acc: 9.562500%
Train Epoch: 5/50 Train_Loss: 2.217921733856201 Train_acc: 14.312500%
Train Epoch: 6/50 Train_Loss: 2.0894062519073486 Train_acc: 25.937500%
Train Epoch: 7/50 Train_Loss: 1.9636449813842773 Train_acc: 42.187500%
Train Epoch: 8/50 Train_Loss: 1.7487901449203491 Train_acc: 62.375000%
Train Epoch: 9/50 Train_Loss: 1.3838627338409424 Train_acc: 77.562500%
Train Epoch: 10/50 Train_Loss: 1.1189829111099243 Train_acc: 82.000000%
Train Epoch: 11/50 Train_Loss: 0.8552320003509521 Train

Train Epoch: 1/50 Train_Loss: 0.007433858700096607 Train_acc: 100.000000%
Train Epoch: 2/50 Train_Loss: 0.004832864739000797 Train_acc: 100.000000%
Train Epoch: 3/50 Train_Loss: 0.013235568068921566 Train_acc: 100.000000%
Train Epoch: 4/50 Train_Loss: 0.005999388638883829 Train_acc: 100.000000%
Train Epoch: 5/50 Train_Loss: 0.009238436818122864 Train_acc: 100.000000%
Train Epoch: 6/50 Train_Loss: 0.005523461848497391 Train_acc: 100.000000%
Train Epoch: 7/50 Train_Loss: 0.004958607256412506 Train_acc: 100.000000%
Train Epoch: 8/50 Train_Loss: 0.0055013070814311504 Train_acc: 100.000000%
Train Epoch: 9/50 Train_Loss: 0.005441001616418362 Train_acc: 100.000000%
Train Epoch: 10/50 Train_Loss: 0.0027462635189294815 Train_acc: 100.000000%
Train Epoch: 11/50 Train_Loss: 0.004911349155008793 Train_acc: 100.000000%
Train Epoch: 12/50 Train_Loss: 0.004884267691522837 Train_acc: 100.000000%
Train Epoch: 13/50 Train_Loss: 0.0030540302395820618 Train_acc: 100.000000%
Train Epoch: 14/50 Train_Loss: 

In [5]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()